In [1]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-06 19:02:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.97MB/s    in 0.5s    

2024-07-06 19:02:01 (1.97 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:200])  


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
print(f"Length of document is {len(text)} character")

Length of document is 1115394 character


In [4]:
# print unique characters in the entire vocabulary
chars = sorted(list(set(text)), reverse=False)
vocab_size = len(chars)
print(f"size of vocabulary is {vocab_size}\nand unique chars are {''.join(chars)}")

size of vocabulary is 65
and unique chars are 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# tokenize
str_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_str = {i:ch for i, ch in enumerate(chars)}
encode = lambda word: [str_to_int[character] for character in word] # takes a string and outputs an interger represnentation for each character in it 
decode = lambda ints: "".join([int_to_str[num] for num in ints]) # takes a list of integers and outputs thier corresponding chars joined in a single string
print(encode("Hii there"))
print(decode(encode("Hii there")))

[20, 47, 47, 1, 58, 46, 43, 56, 43]
Hii there


In [6]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(f"shape is {data.shape}")

shape is torch.Size([1115394])


In [7]:
data[:2000]

tensor([18, 47, 56,  ..., 58,  1, 15])

In [8]:
# split to train and val sets
n = int(0.9*len(data))
train_set = data[:n]
test_set = data[n:]

In [9]:
torch.manual_seed(1337)
BATCH_SIZE = 4 # how many sequency to process at a single pass
BLOCK_SIZE = 8 # max context length to look into for making the predictions

def get_batch(split_type):
    data = train_set if split_type == "train" else test_set

    ix = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])

    return x, y

In [10]:
xb, yb = get_batch("train")
print(f"inputs shape :{xb.shape}")
print(f"inputs:{xb}")
print(f"targets shape :{yb.shape}")
print(f"targets :{yb}")
print("-----------------------------------")
for batch in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[batch,:1+t]
        target = yb[batch,t]
        print(f"when input context is {context.tolist()}the target is {target}")


inputs shape :torch.Size([4, 8])
inputs:tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets shape :torch.Size([4, 8])
targets :tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------------------------
when input context is [24]the target is 43
when input context is [24, 43]the target is 58
when input context is [24, 43, 58]the target is 5
when input context is [24, 43, 58, 5]the target is 57
when input context is [24, 43, 58, 5, 57]the target is 1
when input context is [24, 43, 58, 5, 57, 1]the target is 46
when input context is [24, 43, 58, 5, 57, 1, 46]the target is 43
when input context is [24, 43, 58, 5, 57, 1, 46, 43]the target is 39
when input context is [44]the target is 53
when input context is [44, 53]the target is 56
when in

In [11]:
# create BI-Gram language model
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiGramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super(BiGramLanguageModel, self).__init__()
        
        # embedding module having vocab_size tensors each of length vocab size
        self.embedding_table = nn.Embedding(vocab_size, vocab_size) # each token will selecg a row from this table

    
    def calc_loss(self, y_hat, y_true):
        return F.cross_entropy(y_hat, y_true)

    def forward(self, xb, yb=None):
        # xb shape is BatchSize, Time 
        loss = None
        logits = self.embedding_table(xb) # output is input_shape, embedding Dimension which means B,T, vocabsize
        if yb is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            yb = yb.view(B*T)
            loss = self.calc_loss(logits, yb)
        
        return logits, loss

    def generate(self,idx, max_new_tokens):
        # idx is B,T array of current context

        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)

            # shape now is B,1,C
            logits = logits[:, -1, :] # focus only on the last time step of the context
            probes = F.softmax(logits, dim=1) # diminishes the dim 1 so shape is B,C

            idx_next = torch.multinomial(probes, num_samples=1) # B, 1
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1
        
        return idx


m = BiGramLanguageModel(vocab_size)
logits, loss=m(xb,yb)
print(logits.shape, loss)
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca!UQ!vrpxZQgC-hlkq,ptKqHoiX-jjeLJ &slERj KUsBOL!mpJO!zLg'wNfqHAMgq'hZCWhu.W.IBcP 
RFJ&DEs,nw?pxE?xjNHHVxJ&D&vWWToiERJFuszPyZaNw$
EQJMgzaveDDIoiMl&sMHkzdRptRCPVjwW.RSVMjs-bgRkzrBTEa!!oP fRSxq.PLboTMkX'DUYepxIBFAYuxKXe.jeh
sa!3MGFrSjuM:wX!?BTMl!.?,M:bQzPHpYfN!Cbo'MmtDxBkDD3SBjyFdmY'DOqkWeRjlxyJB-bVbfd&


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
BATCH_SIZE=32
for epoch in range(10000):
    xb,yb = get_batch("train")

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.48514723777771


In [18]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Wr; b R:
TEPro:
ANTExcoth wairudort t whtheay the wissheryoloremestowized S:
Whe moveen aqub k by fe fof pacirill h olave a I mber. de liell, forate o'tad gavisores; el y o ound pal wineatse.
NN gumele caran tr.

Boull ie, IXElitso thiour tf the, tise gematath'elld ourknrdovirou Inavene s

RNGuancern jern:
ORIme pis ne d ltharowid, AUshigounometinindofree thiay mps an y
Joor:
Whareave s s inapre,
BUL: m-be tit sonerayole.
D andu lfo s tangreshat,
KE:
crrser:
Lard-boun lin, h washimammene s joull
